In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

model_dir = '/cluster/work/eliashk/models/qlora_finetuned'

model = AutoPeftModelForCausalLM.from_pretrained(
    model_dir,
    device_map='auto',
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
from random import randint

eval_dataset = load_dataset('json', data_files='/cluster/work/eliashk/semantic-robot/datasets/train.jsonl', split='train')
rand_idx = randint(0, len(eval_dataset))

In [16]:
eval_dataset[0]['messages'][2]['content']

'set instance l0 location|set instance l1 location|set instance l2 location|set instance l3 location|set instance l4 location|set instance o0 portable|set instance o1 portable|set instance o2 portable|set instance o3 portable|set predicate at o0 l3|set predicate at o1 l1|set predicate at o2 l2|set predicate at o3 l0|set predicate is-at l4|set goal at o0 l3|set goal at o1 l0|set goal at o2 l1|set goal at o3 l2|set goal is-at l1|'

In [ ]:

prompt = pipe.tokenizer.apply_chat_template(eval_dataset[0]['messages'][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=4096, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

In [21]:
outputs[0]['generated_text'][len(prompt):].strip()

'Driver1 and Driver2 are at s0, while Truck1 and Truck2 are at s1. Package1 is at s0, and Package2 is at s1. The locations s0 and s1 are connected directly and also through a path via p0-1. Your task is to orchestrate the drivers and trucks so that Driver1 ends up back at s0, Driver2 at s1, Truck1 and Truck2 at s0, with Package1 moved to s1 and Package2 to s0, utilizing the paths and links available between the locations. s0 is linked to s1 and vice versa. There are paths from s0 to p0-1, s1 to p0-1 and vice versa.  \npddl\n(define (domain driverlog) (:requirements :typing) (:types location locatable - object driver truck obj - locatable ) (:predicates (at?obj - locatable?loc - location) (in?obj1 - obj?obj - truck) (driving?d - driver?v - truck) (link?x?y - location) (path?x?y - location) (empty?v - truck) ) (:action LOAD-TRUCK :parameters (?obj - obj?truck - truck?loc - location) :precondition (and (at?truck?loc) (at?obj?loc)) :effect (and (not (at?obj?loc)) (in?obj?truck))) (:action 